In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

In [2]:
#loading the data file
data = pd.read_csv('final_dataset.csv')
data["Date"] = pd.to_datetime(data["Date"])

In [3]:
#checking the data types of our columns
data.dtypes

HomeTeam            object
AwayTeam            object
FTHG               float64
FTAG               float64
FTR                 object
HS                 float64
AS                 float64
HST                float64
AST                float64
HC                 float64
AC                 float64
HY                 float64
AY                 float64
HR                 float64
AR                 float64
Date        datetime64[ns]
dtype: object

In [4]:
# Create the 'Home' rows
home_rows = data.copy()
home_rows['Team'] = home_rows['HomeTeam']  # Team column for home team
home_rows['Venue'] = 'Home'  # Venue column for home team
home_rows['Opponent'] = home_rows['AwayTeam']  # Opponent is the away team
home_rows['GoalsFor'] = home_rows['FTHG']  # Goals scored by home team
home_rows['GoalsAgainst'] = home_rows['FTAG']  # Goals conceded by home team

# Create the 'Away' rows
away_rows = data.copy()
away_rows['Team'] = away_rows['AwayTeam']  # Team column for away team
away_rows['Venue'] = 'Away'  # Venue column for away team
away_rows['Opponent'] = away_rows['HomeTeam']  # Opponent is the home team
away_rows['GoalsFor'] = away_rows['FTAG']  # Goals scored by away team
away_rows['GoalsAgainst'] = away_rows['FTHG']  # Goals conceded by away team

# Combine the two datasets
combined_data = pd.concat([home_rows, away_rows], ignore_index=True)

# Drop unnecessary columns to avoid redundancy
combined_data = combined_data.drop(['HomeTeam', 'AwayTeam', 'FTHG', 'FTAG'], axis=1)

In [5]:
#tokenising the data so its ready for machine learning
combined_data['ftr_num'] = combined_data['FTR'].apply(
    lambda x: 1 if x == 'H' else (0 if x == 'D' else -1)
)
combined_data['Venue_num'] = combined_data['Venue'].apply(lambda x: 1 if x == 'Home' else 0)
combined_data["opp_code"] = combined_data["Opponent"].astype("category").cat.codes
combined_data["day_code"] = combined_data["Date"].dt.dayofweek

In [6]:
#date we're using to seperate the data, we're going to test the latest 2 seasons
season_start_date = pd.Timestamp('2022-08-05')

In [7]:
#we initialise the test data
latest_season = combined_data[combined_data['Date'] >= season_start_date]

In [8]:
latest_season

,FTR,HS,AS,HST,AST,HC,AC,HY,AY,HR,...,Date,Team,Venue,Opponent,GoalsFor,GoalsAgainst,ftr_num,Venue_num,opp_code,day_code
0,A,6.0,17.0,1.0,8.0,6.0,5.0,0.0,0.0,1.0,...,2023-08-11,Burnley,Home,Man City,0.0,3.0,-1,1,17,4.0
1,H,15.0,6.0,7.0,2.0,8.0,3.0,2.0,2.0,0.0,...,2023-08-12,Arsenal,Home,Nott'm Forest,2.0,1.0,1,1,22,5.0
2,D,14.0,16.0,5.0,3.0,10.0,4.0,1.0,4.0,0.0,...,2023-08-12,Bournemouth,Home,West Ham,1.0,1.0,0,1,33,5.0
3,H,27.0,9.0,12.0,3.0,6.0,7.0,2.0,2.0,0.0,...,2023-08-12,Brighton,Home,Luton,4.0,1.0,1,1,16,5.0
4,A,19.0,9.0,9.0,2.0,10.0,4.0,0.0,2.0,0.0,...,2023-08-12,Everton,Home,Fulham,0.0,1.0,-1,1,10,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5316,H,13.0,7.0,6.0,2.0,9.0,3.0,1.0,3.0,0.0,...,2023-05-28,Bournemouth,Away,Everton,0.0,1.0,1,0,9,6.0
5317,A,19.0,11.0,2.0,7.0,12.0,3.0,3.0,0.0,0.0,...,2023-05-28,Tottenham,Away,Leeds,4.0,1.0,-1,0,13,6.0
5318,H,13.0,16.0,4.0,3.0,3.0,5.0,1.0,1.0,0.0,...,2023-05-28,West Ham,Away,Leicester,1.0,2.0,1,0,14,6.0
5319,H,21.0,10.0,8.0,3.0,5.0,4.0,1.0,2.0,0.0,...,2023-05-28,Fulham,Away,Man United,1.0,2.0,1,0,18,6.0


In [9]:
#we initialise the training data
historical_data = combined_data[combined_data['Date'] < season_start_date]

In [10]:
# Prepare training data
X_train = historical_data.drop(['ftr_num', 'Team', 'Opponent', 'Venue', 'Date', 'FTR', 'GoalsFor', 'GoalsAgainst', 'HST', 'AST', 'AS', 'HS', 'HY', 'AY', 'HR', 'AR' ], axis=1)
y_train = historical_data['ftr_num']

In [11]:
# Prepare prediction data
X_test = latest_season.drop(['ftr_num', 'Team', 'Opponent', 'Venue', 'Date', 'FTR', 'GoalsFor', 'GoalsAgainst', 'HST', 'AST', 'AS', 'HS', 'HY', 'AY', 'HR', 'AR' ], axis=1)
y_test = latest_season['ftr_num']

In [12]:
#initialise our random forest classifier
rf = RandomForestClassifier(n_estimators=50, min_samples_split=10, random_state=1)

In [13]:
#here we fit the data
rf.fit(X_train, y_train)

RandomForestClassifier(min_samples_split=10, n_estimators=50, random_state=1)

In [14]:
#we initialise our test prediction
y_test_pred = rf.predict(X_test)

In [15]:
#here we initialise our accuracy
accuracy = accuracy_score(y_test, y_test_pred)

In [16]:
accuracy

0.42697368421052634

In [17]:
# Add predictions to the latest season dataset
latest_season['y_test_pred'] = y_test_pred

# Filter for matches that have been played (where ftr_num is not NaN)
played_matches = latest_season[latest_season['ftr_num'].notna()]

# Display relevant columns: Date, Team, Opponent, Venue, Actual Result, Predicted Result
played_matches_results = played_matches[['Date', 'Team', 'Opponent', 'Venue', 'ftr_num', 'y_test_pred']]

# Rename columns for clarity
played_matches_results = played_matches_results.rename(columns={
    'ftr_num': 'Actual_Result',
    'y_test_pred': 'Predicted_Result'
})





C:\Users\awab\AppData\Local\Temp\ipykernel_25708\2719628863.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  latest_season['y_test_pred'] = y_test_pred


In [18]:
# Display the first few rows of the dataset
played_matches_results

,Date,Team,Opponent,Venue,Actual_Result,Predicted_Result
0,2023-08-11,Burnley,Man City,Home,-1,-1
1,2023-08-12,Arsenal,Nott'm Forest,Home,1,1
2,2023-08-12,Bournemouth,West Ham,Home,0,1
3,2023-08-12,Brighton,Luton,Home,1,-1
4,2023-08-12,Everton,Fulham,Home,-1,1
...,...,...,...,...,...,...
5316,2023-05-28,Bournemouth,Everton,Away,1,1
5317,2023-05-28,Tottenham,Leeds,Away,-1,0
5318,2023-05-28,West Ham,Leicester,Away,1,1
5319,2023-05-28,Fulham,Man United,Away,1,1


In [19]:
#we sort our data by team and date, so its ready to calculate rolling averages
combined_data = combined_data.sort_values(by=['Team', 'Date'])

In [20]:
combined_data

,FTR,HS,AS,HST,AST,HC,AC,HY,AY,HR,...,Date,Team,Venue,Opponent,GoalsFor,GoalsAgainst,ftr_num,Venue_num,opp_code,day_code
4181,D,14.0,3.0,4.0,2.0,7.0,0.0,0.0,0.0,0.0,...,2012-08-18,Arsenal,Home,Sunderland,0.0,0.0,0,1,28,5.0
8761,D,7.0,16.0,4.0,6.0,0.0,11.0,2.0,0.0,0.0,...,2012-08-26,Arsenal,Away,Stoke,0.0,0.0,0,0,27,6.0
8768,A,17.0,11.0,8.0,7.0,10.0,2.0,2.0,2.0,0.0,...,2012-09-02,Arsenal,Away,Liverpool,2.0,0.0,-1,0,15,6.0
4210,H,20.0,9.0,12.0,4.0,8.0,3.0,0.0,0.0,0.0,...,2012-09-15,Arsenal,Home,Southampton,6.0,1.0,1,1,26,5.0
8788,D,14.0,10.0,10.0,4.0,9.0,4.0,1.0,0.0,0.0,...,2012-09-23,Arsenal,Away,Man City,1.0,1.0,0,0,17,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4913,H,20.0,2.0,12.0,1.0,6.0,0.0,0.0,4.0,0.0,...,2024-05-04,Wolves,Away,Man City,1.0,5.0,1,0,17,5.0
363,A,13.0,13.0,7.0,3.0,8.0,3.0,3.0,2.0,0.0,...,2024-05-11,Wolves,Home,Crystal Palace,1.0,3.0,-1,1,8,5.0
4937,H,36.0,4.0,14.0,3.0,10.0,2.0,1.0,1.0,0.0,...,2024-05-19,Wolves,Away,Liverpool,0.0,2.0,1,0,15,6.0
3800,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaT,NaN,Home,NaN,NaN,NaN,-1,1,-1,NaN


In [21]:
#here we calculate rolling averages for wins, home and away shots, home and away shots on target, the goals for and goals against, corners, yellow and red cards

combined_data['is_win'] = (combined_data['ftr_num'] == 1).astype(int)
combined_data['rolling_wins'] = (
    combined_data.groupby('Team')['is_win']   # Group by team
    .rolling(5, min_periods=1)               # Rolling window of last 5 matches
    .mean()                                  # Calculate mean (percentage of wins)
    .reset_index(level=0, drop=True)         # Align with the original DataFrame
)


combined_data['rolling_HS'] = (
    combined_data.groupby('Team')['HS']
    .rolling(5, min_periods=1)
    .mean()
    .reset_index(level=0, drop=True)
)
combined_data['rolling_AS'] = (
    combined_data.groupby('Team')['AS']
    .rolling(5, min_periods=1)
    .mean()
    .reset_index(level=0, drop=True)
)

combined_data['rolling_HST'] = (
    combined_data.groupby('Team')['HST']
    .rolling(5, min_periods=1)
    .mean()
    .reset_index(level=0, drop=True)
)
combined_data['rolling_AST'] = (
    combined_data.groupby('Team')['AST']
    .rolling(5, min_periods=1)
    .mean()
    .reset_index(level=0, drop=True)
)

combined_data['rolling_goals_for'] = (
    combined_data.groupby('Team')['GoalsFor']
    .rolling(5, min_periods=1)
    .mean()
    .reset_index(level=0, drop=True)
)

combined_data['rolling_goals_against'] = (
    combined_data.groupby('Team')['GoalsAgainst']
    .rolling(5, min_periods=1)
    .mean()
    .reset_index(level=0, drop=True)
)

combined_data['rolling_home_corners'] = (
    combined_data.groupby('Team')['HC']
    .rolling(5, min_periods=1)
    .mean()
    .reset_index(level=0, drop=True)
)

combined_data['rolling_away_corners'] = (
    combined_data.groupby('Team')['AC']
    .rolling(5, min_periods=1)
    .mean()
    .reset_index(level=0, drop=True)
)
combined_data['rolling_home_yellowcards'] = (
    combined_data.groupby('Team')['HY']
    .rolling(5, min_periods=1)
    .mean()
    .reset_index(level=0, drop=True)
)

combined_data['rolling_away_yellowcards'] = (
    combined_data.groupby('Team')['AY']
    .rolling(5, min_periods=1)
    .mean()
    .reset_index(level=0, drop=True)
)

combined_data['rolling_home_redcards'] = (
    combined_data.groupby('Team')['HR']
    .rolling(5, min_periods=1)
    .mean()
    .reset_index(level=0, drop=True)
)

combined_data['rolling_away_redcards'] = (
    combined_data.groupby('Team')['AR']
    .rolling(5, min_periods=1)
    .mean()
    .reset_index(level=0, drop=True)
)

In [22]:
combined_data

,FTR,HS,AS,HST,AST,HC,AC,HY,AY,HR,...,rolling_HST,rolling_AST,rolling_goals_for,rolling_goals_against,rolling_home_corners,rolling_away_corners,rolling_home_yellowcards,rolling_away_yellowcards,rolling_home_redcards,rolling_away_redcards
4181,D,14.0,3.0,4.0,2.0,7.0,0.0,0.0,0.0,0.0,...,4.000000,2.00,0.000000,0.00,7.000000,0.000000,0.000000,0.000000,0.0,0.0
8761,D,7.0,16.0,4.0,6.0,0.0,11.0,2.0,0.0,0.0,...,4.000000,4.00,0.000000,0.00,3.500000,5.500000,1.000000,0.000000,0.0,0.0
8768,A,17.0,11.0,8.0,7.0,10.0,2.0,2.0,2.0,0.0,...,5.333333,5.00,0.666667,0.00,5.666667,4.333333,1.333333,0.666667,0.0,0.0
4210,H,20.0,9.0,12.0,4.0,8.0,3.0,0.0,0.0,0.0,...,7.000000,4.75,2.000000,0.25,6.250000,4.000000,1.000000,0.500000,0.0,0.0
8788,D,14.0,10.0,10.0,4.0,9.0,4.0,1.0,0.0,0.0,...,7.600000,4.60,1.800000,0.40,6.800000,4.000000,1.000000,0.400000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4913,H,20.0,2.0,12.0,1.0,6.0,0.0,0.0,4.0,0.0,...,6.600000,5.20,1.000000,2.20,5.800000,3.800000,2.400000,2.200000,0.0,0.2
363,A,13.0,13.0,7.0,3.0,8.0,3.0,3.0,2.0,0.0,...,6.200000,5.00,0.800000,2.40,5.400000,4.200000,2.600000,2.000000,0.0,0.4
4937,H,36.0,4.0,14.0,3.0,10.0,2.0,1.0,1.0,0.0,...,8.400000,3.80,0.800000,2.40,7.200000,4.200000,2.200000,1.800000,0.0,0.6
3800,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
latest_season = combined_data[combined_data['Date'] >= season_start_date]

In [24]:
historical_data = combined_data[combined_data['Date'] < season_start_date]

In [25]:
# training data
X_train = historical_data.drop(['ftr_num', 'Team', 'Opponent', 'Venue', 'Date', 'FTR', 'GoalsFor', 'GoalsAgainst', 'HST', 'AST', 'AS', 'HS', 'HY', 'AY', 'HR', 'AR' ], axis=1)
y_train = historical_data['ftr_num']

In [26]:
# prediction data
X_test = latest_season.drop(['ftr_num', 'Team', 'Opponent', 'Venue', 'Date', 'FTR', 'GoalsFor', 'GoalsAgainst', 'HST', 'AST', 'AS', 'HS', 'HY', 'AY', 'HR', 'AR' ], axis=1)
y_test = latest_season['ftr_num']

In [27]:
rf = RandomForestClassifier(n_estimators=50, min_samples_split=10, random_state=1)

In [28]:
rf.fit(X_train, y_train)

RandomForestClassifier(min_samples_split=10, n_estimators=50, random_state=1)

In [29]:
y_test_pred = rf.predict(X_test)

In [30]:
accuracy = accuracy_score(y_test, y_test_pred)

In [31]:
accuracy

0.7848684210526315